# O PROBLEMA

Imagine agora que você foi contratado(a) como Expert em Data Analytics por um grande hospital para entender como foi o comportamento da população na época da pandemia da COVID-19 e quais indicadores seriam importantes para o planejamento, caso haja um novo surto da doença.

Apesar de ser contratado(a) agora, a sua área observou que a utilização do estudo do PNAD-COVID-19 do IBGE seria uma ótima base para termos boas respostas ao problema proposto, pois são dados confiáveis. Porém, não será necessário utilizar todas as perguntas realizadas na pesquisa para enxergar todas as oportunidades ali postas.

É sempre bom ressaltar que há dados triviais que precisam estar no projeto, pois auxiliam muito na análise dos dados:

## PNAD-COVID 19 do IBGE

O Head de Dados pediu para que você entrasse na base de dados do PNAD-COVID-19 do IBGE e organizasse esta base para análise, utilizando Banco de Dados em Nuvem e trazendo as seguintes características:

- a. Utilização de no máximo 20 questionamentos realizados na pesquisa;
- b. Utilizar 3 meses para construção da solução;
- c. Caracterização dos sintomas clínicos da população;
- d. Comportamento da população na época da COVID-19;
- e. Características econômicas da sociedade;

Seu objetivo será trazer uma breve análise dessas informações, como foi a organização do banco, as perguntas selecionadas para a resposta do problema e quais seriam as principais ações que o hospital deverá tomar em caso de um novo surto de COVID-19.

- Dica: Leiam com atenção a base de dados e toda a documentação que o site o PNAD-COVID-19 traz, principalmente os dicionários, que ajudam e muito no entendimento da Base de Dados.
- Dica 2: Utilizem o que já foi ensinado e consolidado nas outras fases para apresentar a resolução do projeto.

Lembre-se de que você poderá apresentar o desenvolvimento do seu projeto durante as lives com docentes. Essa é uma boa oportunidade para discutir sobre as dificuldades encontradas e pegar dicas valiosas com especialistas e colegas de turma.

Link para a base: https://covid19.ibge.gov.br/pnad-covid/

Outras fontes:

https://www.gov.br/saude/pt-br/assuntos/noticias/2021/janeiro/entenda-a-ordem-de-vacinacao-contra-a-covid-19-entre-os-grupos-prioritarios

https://www.gov.br/saude/pt-br/assuntos/coronavirus/vacinas/esquemas-vacinais

In [2]:
import warnings 
warnings.filterwarnings(action = 'ignore')

In [3]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from pyspark.sql import SparkSession
import findspark
import pandas as pd
import plotly.express as px

### Conectando ao MySQL

In [4]:
'''def mysql_connection(host, user, passwd, database=None):
    engine = create_engine(f'mysql+pymysql://{user}:{passwd}@{host}/{database}')
    return engine.connect()

mysql = mysql_connection('127.0.0.1', 'root', 'admin', 'pnad_covid')'''

"def mysql_connection(host, user, passwd, database=None):\n    engine = create_engine(f'mysql+pymysql://{user}:{passwd}@{host}/{database}')\n    return engine.connect()\n\nmysql = mysql_connection('127.0.0.1', 'root', 'admin', 'pnad_covid')"

### Criando sessão Spark

In [5]:
findspark.init()

In [6]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

23/11/01 00:26:40 WARN Utils: Your hostname, platero-Lenovo-IdeaPad-S145-15IWL resolves to a loopback address: 127.0.1.1; using 192.168.15.109 instead (on interface wlp2s0)
23/11/01 00:26:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/01 00:26:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/01 00:26:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
spark

### Importanto dados tratados da view do arquivo CSV

In [8]:
df = spark.read.csv('dados/dados_exportados/dados_uteis/dados_uteis_gz/2023-10-31_pnad_covid_view.csv.gz', sep=',', inferSchema=True, header=True)
df.show(5, truncate=False)

23/11/01 00:26:55 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+--------------+------------------+-----+------+--------+------------------------------------+-----------------------------+-----------------------+-----------------------------+---------------------------------+--------------------+------------------------+------------------+---------------------------------+--------------------------+---------------------------+-------------------------------+----------------+-------------------------------+------------------------+---------------------+-----------------------+-----------------+-----------------------+--------------------+-------------+-----------------------+-----------+---------------------------+-------------+-----------------+---------------+-----+--------+
|data      |uf            |situacao_domicilio|idade|sexo  |cor_raca|escolaridade                        |questao_estabelecimento_saude|questao_permaneceu_casa|questao_remedio_conta_propria|questao_remedio_orientacao_medica|questao_hospital_SUS|questao_hospital_priva

### Análise exploratória de características comportamentais

In [9]:
df_temp = df.createOrReplaceTempView('df_temp')

#### **<p style='color:gray'> Comportamento dos entrevistados em relação a busca por cuidados médicos</p>**

**Qual foi a quantidade de infectados que buscou atendimento médico?**

In [15]:
df_testes_positivos_estabelecimento_saude = spark.sql(

'''
    SELECT
        count(resultado_teste) AS qtd_testes_positivos,
        CASE
        WHEN questao_estabelecimento_saude = 'Sim' THEN 'Buscou atendimento'
        WHEN questao_estabelecimento_saude = 'Não' THEN 'Não buscou atendimento'
        WHEN questao_estabelecimento_saude = 'Ignorado' THEN 'Ignorado'
        ELSE 'Não aplicável'
        END AS estabelecimento_saude
    from df_temp
    WHERE resultado_teste = 'Positivo'
    GROUP BY estabelecimento_saude

'''
).toPandas()

fig = px.bar(
    data_frame = df_testes_positivos_estabelecimento_saude,
    x = 'estabelecimento_saude',
    y = 'qtd_testes_positivos',
    color= 'estabelecimento_saude',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_testes_positivos': 'Quantidade',
        'estabelecimento_saude': 'Comportamento'
    }
)

fig.update_layout(
    title='<b>Pessoas que buscaram atendimento</b>',
    xaxis_title='Comportamento',
    yaxis_title='Quantidade',
    showlegend=False,
    width=800,
    height=600
)

fig.show()

#### **<p style='color:gray'> Comportamento dos entrevistados em relação a medidas de isolamento social</p>**

**Qual é a porcetagem de pacientes sintomáticos que efetuou medida de isolamento social?**

In [11]:
dicionario = {'Sim': 'Sim', 'Não': 'Não', 'Não aplicável': 'Não', 'Ignorado': 'Não'}

df_sintomaticos_permaneceu_casa = spark.sql(
    '''
        SELECT questao_permaneceu_casa, count(sintoma_covid) as qtd_sintomaticos
        FROM df_temp 
        WHERE sintoma_covid = 'Sim'
        GROUP BY questao_permaneceu_casa
    '''
).toPandas()

df_sintomaticos_permaneceu_casa['questao_permaneceu_casa'] = df_sintomaticos_permaneceu_casa['questao_permaneceu_casa'].map(dicionario)

fig = px.pie(
    data_frame=df_sintomaticos_permaneceu_casa, 
    values = 'qtd_sintomaticos', 
    names = 'questao_permaneceu_casa',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={'
        'qtd_sintomaticos': 'Quantidade',
        'questao_permaneceu_casa': 'Isolamento'
    }
)

fig.update_layout(
    title='<b>Porcentagem de pacientes sintomáticos que adotaram medida de isolamento social</b>',
    legend_title='Legenda',
    width=800, 
    height=600
)

fig.show()

**Qual é a porcetagem de pacientes sintomáticos que efetuou medida de isolamento social por região?**

In [12]:
df_sintomaticos_total_permaneceu_casa_regiao = spark.sql(
    '''
        SELECT regiao, count(sintoma_covid) as qtd_sintomaticos_total
        FROM df_temp 
        WHERE sintoma_covid = 'Sim'
        GROUP BY regiao
    '''
).toPandas()

df_sintomaticos_isolamento_permaneceu_casa_regiao = spark.sql(
    '''
        SELECT regiao, questao_permaneceu_casa, count(sintoma_covid) as qtd_sintomaticos_isolamento
        FROM df_temp 
        WHERE sintoma_covid = 'Sim' AND questao_permaneceu_casa = 'Sim'
        GROUP BY regiao, questao_permaneceu_casa
    '''
).toPandas()

df_permaneceu_casa_regiao = pd.merge(df_sintomaticos_total_permaneceu_casa_regiao, df_sintomaticos_isolamento_permaneceu_casa_regiao, on='regiao')
df_permaneceu_casa_regiao['percentual_sintomaticos_permaneceu_casa'] = ((df_permaneceu_casa_regiao.qtd_sintomaticos_isolamento / df_permaneceu_casa_regiao.qtd_sintomaticos_total) * 100).round(decimals=2)

fig = px.pie(
    data_frame=df_permaneceu_casa_regiao, 
    values = 'percentual_sintomaticos_permaneceu_casa', 
    names = 'regiao',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'percentual_sintomaticos_permaneceu_casa': 'Percentual de pacientes sintomáticos que adotaram medida de isolamento social',
        'regiao': 'Região'
    }
)

fig.update_layout(
    title='<b>Porcentagem de pacientes sintomáticos que adotaram medida de isolamento social por região</b>',
    legend_title='Legenda',
    width=1000, 
    height=600
)

fig.show()

**Qual é a porcetagem de pacientes sintomáticos que efetuou medida de isolamento social por estado?**

In [13]:
df_sintomaticos_total_permaneceu_casa_estado = spark.sql(
    '''
        SELECT uf, count(sintoma_covid) as qtd_sintomaticos_total
        FROM df_temp 
        WHERE sintoma_covid = 'Sim'
        GROUP BY uf
    '''
).toPandas()

df_sintomaticos_isolamento_permaneceu_casa_estado = spark.sql(
    '''
        SELECT uf, questao_permaneceu_casa, count(sintoma_covid) as qtd_sintomaticos_isolamento
        FROM df_temp 
        WHERE sintoma_covid = 'Sim' AND questao_permaneceu_casa = 'Sim'
        GROUP BY uf, questao_permaneceu_casa
    '''
).toPandas()

df_permaneceu_casa_estado = pd.merge(df_sintomaticos_total_permaneceu_casa_estado, df_sintomaticos_isolamento_permaneceu_casa_estado, on='uf')
df_permaneceu_casa_estado['percentual_sintomaticos_permaneceu_casa'] = ((df_permaneceu_casa_estado.qtd_sintomaticos_isolamento / df_permaneceu_casa_estado.qtd_sintomaticos_total) * 100).round(decimals=2)

fig = px.bar(
    data_frame=df_permaneceu_casa_estado.sort_values('percentual_sintomaticos_permaneceu_casa', ascending=False), 
    x='uf',
    y='percentual_sintomaticos_permaneceu_casa',
    color='uf',
    color_discrete_sequence=['#8B0000'],
    labels={
        'percentual_sintomaticos_permaneceu_casa': 'Percentual de pacientes sintomáticos que adotaram medida de isolamento social',
        'uf': 'Estado'
    }
)

fig.update_layout(
    title='<b>Porcentagem de pacientes sintomáticos que adotaram medida de isolamento social por estado</b>',
    yaxis_title='%',
    showlegend=False,
    width=1000, 
    height=600
)

fig.show()